In [2]:
%pylab inline

from xgboost.sklearn import XGBClassifier
from typing import Tuple
import xgboost as xgb
import pandas as pd
import json
import sklearn

dataset = pd.read_csv('data/multisession-eeg.csv')
fromstring = lambda array_str: np.fromstring(array_str, dtype=float, sep=',')
dataset.raw_fft = dataset.raw_fft.apply(fromstring)
# dataset.raw_fft.iloc[0]

Populating the interactive namespace from numpy and matplotlib


C:\Users\riversj\Anaconda3\envs\data-x\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['fromstring']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Passthoughts

What if you could simply *think your password*? That's the premise behind *passthoughts*. We'll discuss passthoughts in more depth in lecture 3, but for now, we'll lay this out as a classification problem:

> Given a reading, and a person, is that person who they claim to be?

We'll structure this problem as follows: For each subject, we'll train a classifier. That subject's readings will be positive example, and everyone else's readings will be negative examples.

We can make this a little fancier by having people use specific thoughts (e.g. "focus on your breathing," "sing a song in your head," etc). We'll make sure our methods can handle this case, but for the time being, we'll just use the `"unabeled"` readings - people doing nothing in particular.

We'll use subject `A` as our "target" individual. We will train on this subject for this assignment, and train against the other subjects in the corpus (subjects `B` and `C`).

In [3]:
def to_matrix (series):
    return np.array([ x for x in series ])

def readings_right_subject_right_task (subj, task, session=0):
    return to_matrix(dataset[
        (dataset['subject'] == subj) &
        (dataset['label'] == task) &
        (dataset['session'] == session)
    ].raw_fft)

def readings_wrong_subj_any_task (subj):
    return to_matrix(dataset[
        (dataset['subject'] != subj)
    ].raw_fft)

In [4]:
dataset['label'].replace(to_replace='breathe_o', value='breatheopen', inplace=True)
dataset[(dataset['subject']=='A') & (dataset['session'] == 0)]['label'].unique()
#dataset['label'].unique()

array(['unlabeled', 'breathe', 'song', 'song_o', 'sport', 'breatheopen',
       'speech', 'face'], dtype=object)

In [5]:
positive = readings_right_subject_right_task('A', 'unlabeled', 0)
negative = readings_wrong_subj_any_task('A')
positive.shape, negative.shape

((40, 516), (1228, 516))

## TODO

Notice how we structured our positive and negative examples:

- *Positive examples*: The right person thinking the right task.

- *Negative examples*: The wrong person thinking any task (whether it is right or wrong).

In the context of passthoughts, consider other possibilites for selecting positive and negative features. Here, (1) pick one configuration of positive and negative examples, aside from the ones listed, and (2) discuss their possible consequences (pros/cons). Explain how you might evaluate this selection (with data, with user experiments, etc - your choice).

- Positive examples: The right person thinking the right task. The wrong person thinking the right task.
- Negative examples: The right person thinking the wrong task.

In the case of a normal password, although it is usually used by one person, the advantage of having a password that is written is that another person can be given the password if they need to use it. If we only look for the right person thinking the right task, we have a two-factor identification system, but it will make it hard or impossible for any other person to be given access.

In [6]:
def readings_right_subject_wrong_task (subj, task, session=0):
    return to_matrix(dataset[
        (dataset['subject'] == subj) &
        (dataset['label'] != task) &
        (dataset['session'] == session)
    ].raw_fft)

positive = np.concatenate((readings_right_subject_right_task('A', 'breatheopen', 0),
                           readings_right_subject_right_task('B', 'breatheopen', 0),
                           readings_right_subject_right_task('C', 'breatheopen', 0)))
negative = readings_right_subject_wrong_task('A', 'breatheopen', 0)
positive.shape, negative.shape

((92, 516), (199, 516))

Now, we'll turn these data into our feature/label matrices `X` and `y`.

In [7]:
X = np.concatenate([positive, negative])

In [8]:
y = np.array([ 0 for x in positive] + [ 1 for x in negative])
assert X.shape[0] == y.shape[0]

Note that we are assigning `0` to "positive" examples, and `1` to "negative" examples. That means `0` will mean "ACCEPT" and `1` will mean "REJECT."

## TODO

Now, train and test a classifier! Estimate your classifier's accuracy.

In [9]:
# Your code here....

def fresh_clf () -> XGBClassifier:
    return XGBClassifier(
        # Don't worry about those parameters for now,
        # though feel free to look them up if you're interested.
        objective= 'binary:logistic',
        seed=27)

def xgb_cross_validate (
    X: np.array,
    y: np.array,
    nfold: int=7
) -> Tuple[XGBClassifier, pd.DataFrame]:
    # eval_metrics:
    # http://xgboost.readthedocs.io/en/latest//parameter.html
    metrics = ['error@0.1', 'auc']
#     metrics = [ 'auc' ]
    # we use the @ syntax to override the default of 0.5 as the threshold for 0 / 1 classification
    # the intent here to to minimize FAR at the expense of FRR
    alg = fresh_clf()
    xgtrain = xgb.DMatrix(X,y)
    param = alg.get_xgb_params()
    cvresults = xgb.cv(param,
                      xgtrain,
                      num_boost_round=alg.get_params()['n_estimators'],
                      nfold=nfold,
                      metrics=metrics,
                      early_stopping_rounds=100
                      )
    alg.set_params(n_estimators=cvresults.shape[0])
    alg.fit(X,y,eval_metric=metrics)
    return alg, cvresults

In [10]:
X_train, X_validate, y_train, y_validate = sklearn.model_selection.train_test_split(
    X, y, 
    test_size=0.33, 
    random_state=42)

clf, cvres = xgb_cross_validate(X_train, y_train)

In [11]:
clf.score(X_validate, y_validate)

0.89690721649484539

For authentication, what we want even more than "accuracy" here are two metrics:

- False Acceptance Rate (FAR): The percentage of readings *not* from subject A incorrectly classified "ACCEPT."
- False Rejection Rate (FRR): The percentage of readings *from* subject A incorrectly classified 'REJECT."

For authentication /security/, we want FAR to be as low as possible (so nobody can break in).
For authentication /usability/, we want FRR to be low (so user's don't get frustrated constantly re-trying their passthought).

In [12]:
def far_frr (classifier, features, labels):
    # predict all the labels
    y_pred = classifier.predict(features)
    false_accepts = 0
    false_rejects = 0
    for predicted, actual in zip(y_pred, labels):
        # if we should have rejected,
        # but in fact accepted,
        if (actual == 1) and (predicted == 0):
            # increment false accepts
            false_accepts += 1
        # if we should have accepted,
        # but in fact rejected,
        if (actual == 0) and (predicted == 1):
            # increment false rejections
            false_rejects += 1
    # calculate proportions for each
    num_intruders = len(list(filter(lambda x: x==1, labels)))
    far = false_accepts / num_intruders if num_intruders > 0 else 0
    num_valids = len(list(filter(lambda x: x==0, labels)))
    frr = false_rejects / num_valids if num_valids > 0 else 0
    return far, frr

In [13]:
far, frr = far_frr(clf, X_validate, y_validate)
f'FAR: {far*100}% - FRR: {frr*100}%'

'FAR: 0.0% - FRR: 28.57142857142857%'

Now, these results might be good. 

But our classifier's accuracy could be misleading.   

Can you see why? 

# Nonstationarity

We are training, and testing, using data recorded over a single session. As we know, EEG changes over time, a property known as *nonstationarity*. Will our great results still hold a few weeks later?

Let's take subject `A`'s data from sessions 1 and 2, which were recorded a few weeks after session 0.

In [14]:
X_subja_sess1 = readings_right_subject_right_task('A', 'unlabeled', 1)
X_subja_sess2 = readings_right_subject_right_task('A', 'unlabeled', 2)
X_subja_later = np.concatenate([X_subja_sess1, X_subja_sess2])
y_subja_later = [ 0 for x in X_subja_later ]
X_subja_later.shape

(616, 516)

Now, let's try the classifier we trained on the original data, testing it on the later data.

In [15]:
far, frr = far_frr(clf, X_subja_later, y_subja_later)
f'FAR: {far*100}% - FRR: {frr*100}%'

'FAR: 0% - FRR: 39.935064935064936%'

As we will discuss more in lecture 3, this is a problem for us. After all, we can calibrate our target subject, but we then expect them to leave the lab and go use the device later on. If their state changes so much that they can no longer be authenticated, we can't very well claim our system is accurate!

## TODO

The crux of the lab focuses on nonstationarity. At minimum, your mission is to quantify and qualify *what* is changing in EEG signals over time. You may use any tools in answering this question.

You also have your choice of corpus:

- Study subject `A`'s recordings over the three sessions provided here.
- Study one subject's recordings over the course of a year.

You can use both of these corpora, if you would like.

Some questions to spur investigation:

- What features of readings cause a classifier that works on earlier recordings fail on later ones?
- What features remain the same? Are there any?
- What might be the source of these changing features? Changing placement in the EEG device? Changing properties of the brain?

Please note below all work you do, and any notes you make along the way. Ideally, your work should read like a story - words (and questions!) interspersed with code. Good luck, and have fun!

### Nonstationarity Exploration

First we need to look at which tasks/passthoughts are actually used in all 3 sessions with A

In [16]:
print('Session 0:', dataset[(dataset['subject']=='A') & (dataset['session'] == 0)]['label'].unique())
print('Session 1:', dataset[(dataset['subject']=='A') & (dataset['session'] == 1)]['label'].unique())
print('Session 2:', dataset[(dataset['subject']=='A') & (dataset['session'] == 2)]['label'].unique())

Session 0: ['unlabeled' 'breathe' 'song' 'song_o' 'sport' 'breatheopen' 'speech'
 'face']
Session 1: ['unlabeled' 'calibration' 'word_x' 'phrase_x' 'face_x' 'breatheopen'
 'song_x' 'sport_x']
Session 2: ['unlabeled' 'calibration' 'breatheclosed' 'word_x' 'word_c' 'phrase_x'
 'phrase_c' 'face_x' 'face_c' 'breatheopen' 'song_x' 'song_c' 'sport_x'
 'sport_c']


Assuming "song" lines up with "song_x" and similarly for "sport" and "face", there are four tasks repeated in all three sessions. For some reason, it seems that the tasks lasted longer in the last session. For example:

In [17]:
print('Session 1:', dataset[(dataset['subject']=='A') & (dataset['session'] == 1) & (dataset['label'] == 'face_x')].shape)
print('Session 2:', dataset[(dataset['subject']=='A') & (dataset['session'] == 2) & (dataset['label'] == 'face_x')].shape)

Session 1: (24, 5)
Session 2: (46, 5)


We note this, but ignore it from here on out.

We now isolate these 4 variables from the dataset. This is in order to avoid having large amounts of data from one session over the others (although as noted above we will still have larger amounts from session 2 than 1 or 0).

In [18]:
dataset['label'].replace(['song', 'sport', 'face'], ['song_x', 'sport_x', 'face_x'], inplace = True)
A_data = dataset[(dataset['label'].isin(['song_x', 'sport_x', 'face_x', 'breatheopen'])) & (dataset['subject'] == 'A')]
A_data.shape

(386, 5)

Now it's time to train our classifier and see how it does if trained across sessions. We can also test it on session 2 after training it on 0 and 1.

In [19]:
def readings_right_subject_right_task (dataset, subj, task, session=0):
    return to_matrix(dataset[
        (dataset['subject'] == subj) &
        (dataset['label'] == task) &
        (dataset['session'] == session)
    ].raw_fft)

def readings_right_subject_wrong_task (dataset, subj, task, session=0):
    return to_matrix(dataset[
        (dataset['subject'] == subj) &
        (dataset['label'] != task) &
        (dataset['session'] == session)
    ].raw_fft)

positive = np.concatenate((readings_right_subject_right_task(A_data, 'A', 'song_x', 0),
                            readings_right_subject_right_task(A_data, 'A', 'song_x', 1),
                            readings_right_subject_right_task(A_data, 'A', 'song_x', 2)))
negative = np.concatenate((readings_right_subject_wrong_task(A_data, 'A', 'song_x', 0),
                           readings_right_subject_wrong_task(A_data, 'A', 'song_x', 1),
                           readings_right_subject_wrong_task(A_data, 'A', 'song_x', 2)))
positive.shape[0] + negative.shape[0]

386

In [20]:
X = np.concatenate([positive, negative])
y = np.array([ 0 for x in positive] + [ 1 for x in negative])
assert X.shape[0] == y.shape[0]

In [21]:
X_train, X_validate, y_train, y_validate = sklearn.model_selection.train_test_split(
    X, y, 
    test_size=0.33, 
    random_state=42)

clf, cvres = xgb_cross_validate(X_train, y_train)

In [22]:
print(clf.score(X_validate, y_validate))
far, frr = far_frr(clf, X_validate, y_validate)
f'FAR: {far*100}% - FRR: {frr*100}%'

0.7109375


'FAR: 5.555555555555555% - FRR: 84.21052631578947%'

This is much worse than I was expecting. It seems like there is very little in common from one session to the next. A somewhat high FRR would not surprise me greatly since there were many more false attempts than true attempts, but an FAR of 5% is much higher than I would expect (especially given how high the FRR is). Clearly, the classifier is not just outright rejecting all attempts, yet it still is managing to reject more than half of the valid attempts. 

Even the overall score (71%) is much lower than we have seen with single session classification. We can try with other labels and see if they perform better.

In [36]:
def Create_pos_neg(A_data, label):
    positive = np.concatenate((readings_right_subject_right_task(A_data, 'A', label, 0),
                                readings_right_subject_right_task(A_data, 'A', label, 1),
                                readings_right_subject_right_task(A_data, 'A', label, 2)))
    negative = np.concatenate((readings_right_subject_wrong_task(A_data, 'A', label, 0),
                               readings_right_subject_wrong_task(A_data, 'A', label, 1),
                               readings_right_subject_wrong_task(A_data, 'A', label, 2)))
    return (positive, negative)

positive, negative = Create_pos_neg(A_data, 'sport_x')
X = np.concatenate([positive, negative])
y = np.array([ 0 for x in positive] + [ 1 for x in negative])
assert X.shape[0] == y.shape[0]
X_train, X_validate, y_train, y_validate = sklearn.model_selection.train_test_split(
    X, y, 
    test_size=0.33, 
    random_state=42)

clf, cvres = xgb_cross_validate(X_train, y_train)
print('For sport_x', clf.score(X_validate, y_validate))
far, frr = far_frr(clf, X_validate, y_validate)
f'FAR: {far*100}% - FRR: {frr*100}%'

For sport_x 0.703125


'FAR: 1.1235955056179776% - FRR: 94.87179487179486%'

In [27]:
positive, negative = Create_pos_neg(A_data, 'face_x')
X = np.concatenate([positive, negative])
y = np.array([ 0 for x in positive] + [ 1 for x in negative])
assert X.shape[0] == y.shape[0]
X_train, X_validate, y_train, y_validate = sklearn.model_selection.train_test_split(
    X, y, 
    test_size=0.33, 
    random_state=42)

clf, cvres = xgb_cross_validate(X_train, y_train)
print('For face_x', clf.score(X_validate, y_validate))
far, frr = far_frr(clf, X_validate, y_validate)
f'FAR: {far*100}% - FRR: {frr*100}%'

For face_x 0.8046875


'FAR: 4.3478260869565215% - FRR: 58.333333333333336%'

In [31]:
positive, negative = Create_pos_neg(A_data, 'breatheopen')
X = np.concatenate([positive, negative])
y = np.array([ 0 for x in positive] + [ 1 for x in negative])
assert X.shape[0] == y.shape[0]
X_train, X_validate, y_train, y_validate = sklearn.model_selection.train_test_split(
    X, y, 
    test_size=0.33, 
    random_state=42)

clf, cvres = xgb_cross_validate(X_train, y_train)
print('For breatheopen', clf.score(X_validate, y_validate))
far, frr = far_frr(clf, X_validate, y_validate)
f'FAR: {far*100}% - FRR: {frr*100}%'

For breatheopen 0.7265625


'FAR: 2.1739130434782608% - FRR: 91.66666666666666%'

The classifier seems to perform poorly for all of these labels. This seems to indicate that the nonstationarity, at least in this data sample, is so bad that the same classifier will not work across multiple sessions. 

One thing to keep in mind is that all imposter attempts in this scenario were from subject A, so there is a possibility that training the classifier against other subjects would improve the scores. We essentially only had one factor of identification for this scenario which was the knowledge factor.

In [41]:
def fresh_clf () -> XGBClassifier:
    return XGBClassifier(
        # Don't worry about those parameters for now,
        # though feel free to look them up if you're interested.
        objective= 'binary:logistic',
        seed=27)

def xgb_cross_validate (
    X: np.array,
    y: np.array,
    nfold: int=7
) -> Tuple[XGBClassifier, pd.DataFrame]:
    # eval_metrics:
    # http://xgboost.readthedocs.io/en/latest//parameter.html
    metrics = ['error@0.9']
#     metrics = [ 'auc' ]
    # we use the @ syntax to override the default of 0.5 as the threshold for 0 / 1 classification
    # the intent here to to minimize FAR at the expense of FRR
    alg = fresh_clf()
    xgtrain = xgb.DMatrix(X,y)
    param = alg.get_xgb_params()
    cvresults = xgb.cv(param,
                      xgtrain,
                      num_boost_round=alg.get_params()['n_estimators'],
                      nfold=nfold,
                      metrics=metrics,
                      early_stopping_rounds=100
                      )
    alg.set_params(n_estimators=cvresults.shape[0])
    alg.fit(X,y,eval_metric=metrics)
    return alg, cvresults

In [42]:
positive, negative = Create_pos_neg(A_data, 'face_x')
X = np.concatenate([positive, negative])
y = np.array([ 0 for x in positive] + [ 1 for x in negative])
assert X.shape[0] == y.shape[0]
X_train, X_validate, y_train, y_validate = sklearn.model_selection.train_test_split(
    X, y, 
    test_size=0.33, 
    random_state=42)

clf, cvres = xgb_cross_validate(X_train, y_train)
print('For face_x', clf.score(X_validate, y_validate))
far, frr = far_frr(clf, X_validate, y_validate)
f'FAR: {far*100}% - FRR: {frr*100}%'

For face_x 0.8046875


'FAR: 4.3478260869565215% - FRR: 58.333333333333336%'